In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications.resnet50 import ResNet50,preprocess_input,decode_predictions
from keras.layers import Dense,Conv2D,Input,Dropout,MaxPooling2D,Flatten,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.models import Sequential,Model
from keras.utils import np_utils
import random
import cv2

Using TensorFlow backend.


In [2]:
dirs = os.listdir("./men-women-classification/")
print(dirs)

['men', 'women']


In [3]:
path = "./men-women-classification/"

In [4]:
images = []
labels = []

labels_dict = {'men':0,'women':1}
dict_labels =  { 0:"men",1:"women"}

In [5]:
for ix in dirs:
    class_path = path + ix + "/"
    img_names = os.listdir(class_path)
    for im in img_names:
        im = image.load_img(class_path + im, target_size=(224,224))
        im_array = image.img_to_array(im)
        images.append(im_array)
        labels.append(labels_dict[ix])
        
print (len(images), len(labels))

1000 1000


In [6]:
combined = list(zip(images,labels))
random.shuffle(combined)

images[:],labels[:] = zip(*combined) 

In [7]:
X_train = np.array(images)
y_train = np.array(labels)
y_train = np_utils.to_categorical(y_train)
print (X_train.shape, y_train.shape)

(1000, 224, 224, 3) (1000, 2)


In [8]:
res_model = ResNet50(include_top = False , weights = 'imagenet' , input_shape = (224,224,3))

W0728 17:42:03.904074 140521947629376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 17:42:04.210152 140521947629376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 17:42:04.318412 140521947629376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0728 17:42:04.438638 140521947629376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0728 17:42:04.439691

In [9]:
res_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [10]:
avg = GlobalAveragePooling2D()(res_model.output)
fc1 = Dense(256, activation='relu')(avg)
fc2 = Dense(2, activation='softmax')(fc1)

model = Model(inputs=res_model.inputs, outputs=fc2)
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
for ix,layers in enumerate(model.layers):
    print (ix, layers)

0 <keras.engine.input_layer.InputLayer object at 0x7fcd75ec60f0>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7fcd75ec66a0>
2 <keras.layers.convolutional.Conv2D object at 0x7fcd75ec67b8>
3 <keras.layers.normalization.BatchNormalization object at 0x7fcd75ec6be0>
4 <keras.layers.core.Activation object at 0x7fcd75ec6dd8>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7fcd51f5ee48>
6 <keras.layers.pooling.MaxPooling2D object at 0x7fcd51efef28>
7 <keras.layers.convolutional.Conv2D object at 0x7fcd75d99f28>
8 <keras.layers.normalization.BatchNormalization object at 0x7fcd5062b3c8>
9 <keras.layers.core.Activation object at 0x7fcd5062bcc0>
10 <keras.layers.convolutional.Conv2D object at 0x7fcd5061b630>
11 <keras.layers.normalization.BatchNormalization object at 0x7fcd505f8da0>
12 <keras.layers.core.Activation object at 0x7fcd505bb748>
13 <keras.layers.convolutional.Conv2D object at 0x7fcd50512fd0>
14 <keras.layers.convolutional.Conv2D object at 0x7fcd504b5940>
15 <keras.lay

In [12]:
for ix in range(171):
    model.layers[ix].trainable = False

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [14]:
adam = Adam(lr=0.0003)

In [15]:
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

W0728 17:42:16.628257 140521947629376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [16]:
model.fit(X_train,y_train,epochs=5,shuffle=True,batch_size=64,validation_split=.20) 

W0728 17:42:16.857084 140521947629376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 800 samples, validate on 200 samples
Epoch 1/5
800/800 [==============================] - 134s 167ms/step - loss: 0.6147 - acc: 0.7175 - val_loss: 0.6249 - val_acc: 0.7350
Epoch 2/5
800/800 [==============================] - 133s 167ms/step - loss: 0.1971 - acc: 0.9325 - val_loss: 0.3149 - val_acc: 0.8950
Epoch 3/5
800/800 [==============================] - 138s 172ms/step - loss: 0.0952 - acc: 0.9800 - val_loss: 0.3137 - val_acc: 0.8900
Epoch 4/5
800/800 [==============================] - 147s 184ms/step - loss: 0.0583 - acc: 0.9913 - val_loss: 0.3103 - val_acc: 0.8850
Epoch 5/5
800/800 [==============================] - 150s 187ms/step - loss: 0.0327 - acc: 1.0000 - val_loss: 0.3305 - val_acc: 0.8900


In [19]:
img_path = '/home/vandit/Desktop/My Projects/My_Submissions/Men_Women_Classification/men-women-classification/women/Sophie.jpg'
img = image.load_img(img_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
ans = model.predict(x)
print(dict_labels[np.argmax(ans)])

(1, 224, 224, 3)
women
